In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from poliastro.bodies import Moon 
from scipy.linalg import block_diag
from filterpy.common import Saver
from astropy import units as u
from poliastro.twobody.propagation import cowell as cowell
from poliastro.twobody.propagation import farnocchia as farnocchia
from poliastro.twobody.propagation import vallado as vallado
from poliastro.twobody.propagation import mikkola as mikkola
from poliastro.twobody.propagation import markley as markley
from poliastro.twobody.propagation import pimienta as pimienta
from poliastro.twobody.propagation import gooding as gooding
from poliastro.twobody.propagation import danby as danby
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

In [2]:
############################################
dt = 10
############################################
df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

real_X, real_Y, real_Z = [], [], []
for i in range(len(df)):
    altitude = real_Altitudes[i]
    latitude = real_Latitudes[i]
    longitude = real_Longitudes[i]
    x, y, z = spherical2cartesian(altitude, latitude, longitude)
    real_X.append(x)
    real_Y.append(y)
    real_Z.append(z)
real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)

In [3]:
# Initial State Vector:
x, y, z = real_X[0], real_Y[0], real_Z[0]
vx, vy, vz = real_Vxs[0], real_Vys[0], real_Vzs[0]

In [4]:
FILE = pd.read_csv('Pos_Err_d100_quartile_40-60.csv')
H = FILE['H']
LAT = FILE['LAT']
LON = FILE['LON']

In [5]:
def find_F_matrix(x:float,y:float,z:float)-> np.array:
    upleft = np.eye(3)
    bottleft = np.zeros((3,3))
    # bottleft = np.eye(3)
    bottright = np.eye(3)

    mi = 4.9048695e3 # km^3/s^2 	
    r = np.sqrt( x**2 + y**2 + z**2)


    J = np.zeros((3,3))
    # First Row
    J[0,0] = (3*mi*x**2/r**5) - mi/r**3
    J[0,1] = 3*mi*x*y/r**5
    J[0,2] = 3*mi*x*z/r**5 
    # Second Row
    J[1,0] = 3*mi*y*x/r**5
    J[1,1] = (3*mi*y**2/r**5) - mi/r**3 
    J[1,2] = 3*mi*y*z/r**5 
    # Third Row
    J[2,0] = 3*mi*z*x/r**5
    J[2,1] = 3*mi*z*y/r**5 
    J[2,2] = (3*mi*z**2/r**5) - mi/r**3 
    # End
    upright = J
    # Bulk
    tmp1 = np.hstack([upleft,upright])
    tmp2 = np.hstack([bottleft,bottright])
    phi = np.vstack([tmp1,tmp2])
    return phi

# Init

In [6]:
kf = KalmanFilter(dim_x = 6, dim_z = 3)
iX, iY, iZ, iVx, iVy, iVz  = 0,1,2,3,4,5

kf.F = find_F_matrix(x, y, z)

q1 = np.eye(3)*1e1*0.25
# q1 = np.eye(3)*1e1*0.25


q2 = np.eye(3)*1e-1*0.01

kf.Q = block_diag(q1, q2)
# Measurement Matrix
kf.H[iX, iX] = 1
kf.H[iZ, iZ] = 1
kf.H[iY, iY] = 1
kf.R *= 250  # KM 
kf.x = np.array([[x, y, z, vx, vy, vz]]).T
kf.u = 0

kf.P = np.eye(6) * 1000
kf.P[2,2]=100
kf.P[5,5]=100

saver = Saver(kf)

In [7]:
kf.P

array([[1000.,    0.,    0.,    0.,    0.,    0.],
       [   0., 1000.,    0.,    0.,    0.,    0.],
       [   0.,    0.,  100.,    0.,    0.,    0.],
       [   0.,    0.,    0., 1000.,    0.,    0.],
       [   0.,    0.,    0.,    0., 1000.,    0.],
       [   0.,    0.,    0.,    0.,    0.,  100.]])

# Predictor - Corrector

In [8]:
mus = []
Zs = []
for idx in range(670):
########################################################################
    # PREDICTOR:
    x,y,z = kf.x[0],kf.x[1],kf.x[2] 
    vx,vy,vz = kf.x[3],kf.x[4],kf.x[5]
    r = np.array([x,y,z])*u.km
    v = np.hstack([vx,vy,vz])*u.km/u.s
    tofs = np.linspace(10,10,1)*u.s # 10seconds
    # Propagate:
    rr, vv = cowell(Moon.k, r, v, tofs=tofs )
    # Calculate State Transition Matrix:
    foo  = rr.to_value()[0]
    foo2 = vv.to_value()[0]
    kf.F = find_F_matrix(foo[0],foo[1],foo[2])
    # Predict:
    kf.predict() # P
    kf.x = np.array([foo[0],foo[1],foo[2], foo2[0], foo2[1], foo2[2]  ])
########################################################################
    # CORRECTOR
    if H[idx] > 0:
        XX,YY,ZZ = spherical2cartesian(H[idx],LAT[idx],LON[idx]) # measurement
        Z = np.array([XX,YY,ZZ])
        kf.update(Z)
        saver.save()
        Zs.append(Z)
        XX,YY,ZZ = 0, 0, 0  # reset
    else:
        saver.save()
        Zs.append(np.array([np.NaN,np.NaN,np.NaN]))
    mus.append(kf.x)

In [9]:
x_pred = []
for mu in mus:
    x = mu[0]
    x_pred.append(x)
x_true = real_X[:len(x_pred)]

y_pred = []
for mu in mus:
    y = mu[1]
    y_pred.append(y)
y_true = real_Y[:len(y_pred)]

z_pred = []
for mu in mus:
    z = mu[2]
    z_pred.append(z)
z_true = real_Z[:len(z_pred)]
####################################################################
######################## - MEAS - ##################################
X_m = []
for i in range(len(Zs)):
    X_m.append(Zs[i][0])

Y_m = []
for i in range(len(Zs)):
    Y_m.append(Zs[i][1])

Z_m = []
for i in range(len(Zs)):
    Z_m.append(Zs[i][2])

In [10]:
X = saver.x
P = saver.P
P_arr = np.array(P)
Pxx, Pyy, Pzz = [], [], []
for i in range(len(P)):
    Pxx.append(P[i][0,0])
    Pyy.append(P[i][1,1])
    Pzz.append(P[i][2,2])
Pxx, Pyy, Pzz = np.array(Pxx), np.array(Pyy), np.array(Pzz)
Pxx, Pyy, Pzz = np.sqrt(Pxx), np.sqrt(Pyy), np.sqrt(Pzz)

In [11]:
lw = 1
S =3

plt.figure(dpi=250, tight_layout=True)
# plt.plot(Pxx, '--r', linewidth=lw)
# plt.plot(-Pxx, '--r', linewidth=lw)
plt.plot(x_pred, '-k', linewidth=lw)
plt.plot(x_true, 'r', linewidth=lw)
plt.scatter(range(len(X_m)),X_m, s=S)
# plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(x_true)//2])
# plt.ylim([-550, 550])
plt.ylabel('X [Km]')
plt.show()

plt.figure(dpi=250, tight_layout=True)
# plt.plot(Pxx, '--r', linewidth=lw)
# plt.plot(-Pxx, '--r', linewidth=lw)
plt.plot(y_pred, '-k', linewidth=lw)
plt.plot(y_true, 'r', linewidth=lw)
plt.scatter(range(len(Y_m)),Y_m, s=S)
# plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(x_true)//2])
plt.ylabel('Y [Km]')
plt.show()

plt.figure(dpi=250, tight_layout=True)
# plt.plot(Pxx, '--r', linewidth=lw)
# plt.plot(-Pxx, '--r', linewidth=lw)
plt.plot(z_pred, '-k', linewidth=lw)
plt.plot(z_true, 'r', linewidth=lw)
plt.scatter(range(len(Z_m)),Z_m, s=S)
# plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(x_true)//2])
plt.ylabel('Z [Km]')
plt.show()

# Errors 

In [12]:
plt.figure(dpi=180, tight_layout=True, figsize=(10,10))
lw = 0.8

plt.subplot(311)
x_pred = np.array(x_pred)
x_true = np.array(x_true)
diff = []
for x,y in zip(x_pred,x_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Error along X (LCLF)')
plt.plot(diff, '-k', linewidth=lw)
plt.ylim([-0.2,2])
plt.xlim([0,620])
plt.ylabel('Km')
plt.show()

plt.subplot(312)
y_pred = np.array(y_pred)
y_true = np.array(y_true)
diff = []
for x,y in zip(y_pred,y_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Error along Y (LCLF)')
plt.plot(Pyy, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.xlim([0,620])
plt.ylim([-0.2,2])
plt.ylabel('Km')
plt.show()

plt.subplot(313)
z_pred = np.array(z_pred)
z_true = np.array(z_true)
diff = []
for x,y in zip(z_pred,z_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Error along Z (LCLF)')
plt.xlim([0,620])
plt.ylim([-0.2,2])
plt.plot(Pzz, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Km')
plt.show()

In [13]:
plt.figure(dpi=180, tight_layout=True, figsize=(10,10))
lw = 0.8

plt.subplot(311)
x_pred = np.array(x_pred)
x_true = np.array(x_true)
diff = []
for x,y in zip(x_pred,x_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Error along X (LCLF)')
plt.plot(Pxx, '--r', linewidth=lw)
plt.plot(-Pxx, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.legend(['$\pm\sigma$','_nolegend_','error on KF estimation'], loc='upper right')
plt.ylim([-70,70])
plt.xlim([0,620])
plt.ylabel('Km')
plt.show()

plt.subplot(312)
y_pred = np.array(y_pred)
y_true = np.array(y_true)
diff = []
for x,y in zip(y_pred,y_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Error along Y (LCLF)')
plt.plot(Pyy, '--r', linewidth=lw)
plt.plot(-Pyy, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.xlim([0,620])
plt.ylim([-70,70])
plt.legend(['$\pm\sigma$','_nolegend_','error on KF estimation'], loc='upper right')
plt.ylabel('Km')
plt.show()

plt.subplot(313)
z_pred = np.array(z_pred)
z_true = np.array(z_true)
diff = []
for x,y in zip(z_pred,z_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Error along Z (LCLF)')
plt.xlim([0,620])
plt.ylim([-70,70])
plt.plot(Pzz, '--r', linewidth=lw)
plt.plot(-Pzz, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.legend(['$\pm\sigma$','_nolegend_','error on KF estimation'], loc='upper right')
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Km')
plt.show()

# Plotting: X coordinate - LCLC frame

In [14]:
lw = 1
S =3

plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(x_true, 'r', linewidth=lw)
plt.plot(Pxx+x_true, '--r', linewidth=lw)
plt.plot(-Pxx+x_true, '--r', linewidth=lw)
plt.plot(x_pred, '-k', linewidth=lw)
plt.scatter(range(len(X_m)),X_m, s=S)
plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([0, len(x_true)//2])
plt.ylabel('X [Km]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, :Img.shape[1]//2 ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()

plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(x_true, 'r', linewidth=lw)
plt.plot( Pxx+x_true, '--r', linewidth=lw)
plt.plot(-Pxx+x_true, '--r', linewidth=lw)
plt.plot(x_pred, '-k', linewidth=lw)
plt.scatter(range(len(X_m)),X_m, s=S)
plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([len(x_true)//2, len(x_true)])
plt.ylabel('X [Km]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()

# Y LCLF

In [15]:
plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(y_true, 'r', linewidth=lw)
plt.plot(Pyy+y_true, '--r', linewidth=lw)
plt.plot(-Pyy+y_true, '--r', linewidth=lw)
plt.plot(y_pred, '-k', linewidth=lw)
plt.scatter(range(len(Y_m)),Y_m, s=S)
plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([0, len(y_true)//2])
# plt.ylim([-5e7,5e7])
plt.ylabel('Y [Km]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, :Img.shape[1]//2 ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()



plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(y_true, 'r', linewidth=lw)
plt.plot( Pyy+y_true, '--r', linewidth=lw)
plt.plot(-Pyy+y_true, '--r', linewidth=lw)
plt.plot(y_pred, '-k', linewidth=lw)
plt.scatter(range(len(Y_m)),Y_m, s=S)
plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([len(y_true)//2, len(y_true)])
plt.ylabel('Y [Km]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()

# Z LCLF

In [16]:
plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(z_true, 'r', linewidth=lw)
plt.plot(Pzz+z_true, '--r', linewidth=lw)
plt.plot(-Pzz+z_true, '--r', linewidth=lw)
plt.plot(z_pred, '-k', linewidth=lw)
plt.scatter(range(len(Z_m)),Z_m, s=S)
plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([0, len(z_true)//2])
plt.ylabel('Z [Km]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, :Img.shape[1]//2 ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()



plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(z_true, 'r', linewidth=lw)
plt.plot( Pzz+z_true, '--r', linewidth=lw)
plt.plot(-Pzz+z_true, '--r', linewidth=lw)
plt.plot(z_pred, '-k', linewidth=lw)
plt.scatter(range(len(Z_m)),Z_m, s=S)
plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([len(z_true)//2, len(z_true)])
plt.ylabel('Z [Km]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()

# Velocities

In [17]:
X = saver.x
X_arr = np.array(X)
Vxx, Vyy, Vzz = [], [], []
for i in range(len(X)):
    Vxx.append(X[i][3])
    Vyy.append(X[i][4])
    Vzz.append(X[i][5])
Vxx, Vyy, Vzz = np.array(Vxx), np.array(Vyy), np.array(Vzz)


plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(real_Vxs, 'r', linewidth=lw)
# plt.plot(Pzz+Vxx, '--r', linewidth=lw)
# plt.plot(-Pzz+Vxx, '--r', linewidth=lw)
plt.plot(Vxx, '-k', linewidth=lw)
# plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
plt.legend(['True Velocity','Velocity estimated by KF'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([0, len(real_Vxs)//2])
plt.ylabel('Vx [Km/s]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, :Img.shape[1]//2 ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()



plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(real_Vxs, 'r', linewidth=lw)
# plt.plot( Pzz+z_true, '--r', linewidth=lw)
# plt.plot(-Pzz+z_true, '--r', linewidth=lw)
plt.plot(Vxx, '-k', linewidth=lw)
plt.legend(['True Velocity','Velocity estimated by KF'])
# plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([len(Vxx)//2, len(Vxx)])
plt.ylabel('Vx [Km/s]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()

In [18]:
plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(real_Vys, 'r', linewidth=lw)
# plt.plot(Pzz+Vxx, '--r', linewidth=lw)
# plt.plot(-Pzz+Vxx, '--r', linewidth=lw)
plt.plot(Vyy, '-k', linewidth=lw)
# plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
plt.legend(['True Velocity','Velocity estimated by KF'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([0, len(real_Vys)//2])
plt.ylabel('Vy [Km/s]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, :Img.shape[1]//2 ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()



plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(real_Vys, 'r', linewidth=lw)
# plt.plot( Pzz+z_true, '--r', linewidth=lw)
# plt.plot(-Pzz+z_true, '--r', linewidth=lw)
plt.plot(Vyy, '-k', linewidth=lw)
plt.legend(['True Velocity','Velocity estimated by KF'])
# plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([len(Vyy)//2, len(Vyy)])
plt.ylabel('Vy [Km/s]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()

In [19]:
plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(real_Vzs, 'r', linewidth=lw)
# plt.plot(Pzz+Vxx, '--r', linewidth=lw)
# plt.plot(-Pzz+Vxx, '--r', linewidth=lw)
plt.plot(Vzz, '-k', linewidth=lw)
# plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
plt.legend(['True Velocity','Velocity estimated by KF'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([0, len(real_Vys)//2])
plt.ylabel('Vz [Km/s]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, :Img.shape[1]//2 ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()



plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
plt.plot(real_Vzs, 'r', linewidth=lw)
# plt.plot( Pzz+z_true, '--r', linewidth=lw)
# plt.plot(-Pzz+z_true, '--r', linewidth=lw)
plt.plot(Vzz, '-k', linewidth=lw)
plt.legend(['True Velocity','Velocity estimated by KF'])
# plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
plt.xlabel(f'Step Size: {dt}')
plt.xlim([len(Vzz)//2, len(Vzz)])
plt.ylabel('Vz [Km/s]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()